table of contents


# Import

## Packages

In [1]:
#import basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import for time 
import dask.dataframe as dd
import os
from tqdm import tqdm
import time

#import pyarrow -> needed for feather
import pyarrow

## Straight importing files (but see below for dealing with huge CSV file

In [2]:
# %%time

# #import dataset - but it is a big file. see below for this case. 
# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")

# Case of Huge CSV File

## Readlines

In [3]:
%%time

# %%time will print out the CPU time and wall time 
print('hello')

hello
CPU times: user 242 µs, sys: 100 µs, total: 342 µs
Wall time: 499 µs


In [4]:
#path for train dataset
train_path = '../NYCTaxi/train.csv'

In [5]:
%%time 

#want to know the exact number of rows from the large file -> this way is a lot shorter to find how big the data is. 

#method 1 : use file.readlines 
with open(train_path) as file:
    n_rows = len(file.readlines())

print(f'Exact number of rows: {n_rows}')

Exact number of rows: 55423857
CPU times: user 19.2 s, sys: 31.6 s, total: 50.9 s
Wall time: 1min 12s


In [6]:
%%time 

#take a look at the train dataset
test_temp = pd.read_csv(train_path, nrows= 10)
test_temp.head()

CPU times: user 9.55 ms, sys: 23.5 ms, total: 33.1 ms
Wall time: 62.5 ms


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [7]:
#info on temp table -> to see column datatype
test_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                10 non-null     object 
 1   fare_amount        10 non-null     float64
 2   pickup_datetime    10 non-null     object 
 3   pickup_longitude   10 non-null     float64
 4   pickup_latitude    10 non-null     float64
 5   dropoff_longitude  10 non-null     float64
 6   dropoff_latitude   10 non-null     float64
 7   passenger_count    10 non-null     int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 768.0+ bytes


## Using Chunksize and load chunks to list

In [8]:
#change the column type to optimize for memory usage
traintypes = {
                    'fare_amount': 'float32',
                    'pickup_datetime': 'str',
                    'pickup_longitude': 'float32',
                    'pickup_latitude': 'float32',
                    'dropoff_longitude': 'float32',
                    'dropoff_latitude': 'float32',
                    'passenger_count': 'uint8'
}

#to get the column names, we can use .keys() and save it as cols. 
cols = list(traintypes.keys())

In [9]:
#make a chunk size to run 5mil rows at each chunk
chunksize = 5_000_000 
#here, _ is used as a visual separator. 

In [10]:
%%time
#make an empty list to start with
df_list = []

#for loop using chunksize above. 
for df_chunk in tqdm(pd.read_csv(train_path, usecols = cols, dtype=traintypes, chunksize = chunksize)):
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0,16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc = True, format = '%Y-%m-%d %H:%M')
    
    #append the chunk to list and merge all
    df_list.append(df_chunk)
    
#12 iterations. 

12it [01:54,  9.54s/it]

CPU times: user 1min 34s, sys: 12.5 s, total: 1min 47s
Wall time: 1min 54s


In [11]:
#end result of df_list. -> each iteration results 5M chunk. 
df_list

[         fare_amount           pickup_datetime  pickup_longitude  \
 0                4.5 2009-06-15 17:26:00+00:00        -73.844315   
 1               16.9 2010-01-05 16:52:00+00:00        -74.016045   
 2                5.7 2011-08-18 00:35:00+00:00        -73.982735   
 3                7.7 2012-04-21 04:30:00+00:00        -73.987129   
 4                5.3 2010-03-09 07:51:00+00:00        -73.968094   
 ...              ...                       ...               ...   
 4999995         16.5 2011-01-24 21:33:00+00:00        -74.003883   
 4999996          9.0 2013-10-11 12:12:00+00:00        -73.995102   
 4999997         10.5 2014-12-06 23:04:00+00:00        -73.981064   
 4999998         10.0 2015-05-30 19:01:00+00:00        -73.965401   
 4999999          4.9 2012-07-11 08:12:00+00:00        -73.972595   
 
          pickup_latitude  dropoff_longitude  dropoff_latitude  passenger_count  
 0              40.721317         -73.841614         40.712276                1  
 1    

In [12]:
#merge all dataframe lists into one dataframe
trainm_df = pd.concat(df_list)

#delete the dataframe list to release memory
del df_list

#check trainm_df
trainm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
 #   Column             Dtype              
---  ------             -----              
 0   fare_amount        float32            
 1   pickup_datetime    datetime64[ns, UTC]
 2   pickup_longitude   float32            
 3   pickup_latitude    float32            
 4   dropoff_longitude  float32            
 5   dropoff_latitude   float32            
 6   passenger_count    uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB


In [13]:
display(trainm_df.head())
display(trainm_df.tail(10))

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:00+00:00,-73.844315,40.721317,-73.841614,40.712276,1
1,16.9,2010-01-05 16:52:00+00:00,-74.016045,40.711304,-73.979271,40.782005,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2
3,7.7,2012-04-21 04:30:00+00:00,-73.987129,40.733143,-73.991570,40.758091,1
4,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
55423846,11.7,2010-05-28 07:49:00+00:00,-73.947159,40.780228,-73.976807,40.758865,2
55423847,6.1,2011-09-16 00:46:00+00:00,-73.990944,40.736618,-73.992294,40.740314,3
55423848,6.0,2013-05-24 00:13:00+00:00,-73.993484,40.747372,-73.998436,40.730461,1
55423849,12.0,2014-03-04 22:25:00+00:00,-73.983017,40.745083,-73.954178,40.767338,1
55423850,4.5,2015-03-22 16:37:00+00:00,-73.981056,40.737457,-73.985474,40.729298,1
55423851,14.0,2014-03-15 03:28:00+00:00,-74.005272,40.740028,-73.963280,40.762554,1
55423852,4.2,2009-03-24 20:46:00+00:00,-73.957787,40.765530,-73.951637,40.773960,1
55423853,14.1,2011-04-02 22:04:00+00:00,-73.970505,40.752323,-73.960541,40.797340,1
55423854,28.9,2011-10-26 05:57:00+00:00,-73.980904,40.764629,-73.870605,40.773964,1
55423855,7.5,2014-12-12 11:33:00+00:00,-73.969719,40.797668,-73.970886,40.783314,1


## Feather

In [14]:
%%time
#save it into feather format ->  fast, lightweight, easy to use binary file format for storing dataframes. 

trainm_df.to_feather('nyc_taxi_data_raw.feather')

CPU times: user 2.33 s, sys: 860 ms, total: 3.19 s
Wall time: 2.52 s


In [15]:
%%time
#load the same dataframe next time directly, without reading the csv file again
df = pd.read_feather('nyc_taxi_data_raw.feather')

CPU times: user 1.3 s, sys: 4.73 s, total: 6.03 s
Wall time: 8.41 s


In [16]:
%%time
#to verify we have loaded the saved dataframe of 55 mil rows. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
 #   Column             Dtype              
---  ------             -----              
 0   fare_amount        float32            
 1   pickup_datetime    datetime64[ns, UTC]
 2   pickup_longitude   float32            
 3   pickup_latitude    float32            
 4   dropoff_longitude  float32            
 5   dropoff_latitude   float32            
 6   passenger_count    uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB
CPU times: user 11.6 ms, sys: 13.7 ms, total: 25.3 ms
Wall time: 251 ms


# Data Cleaning

## Fare_amount

In [17]:
#describe for understanding the data
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542386e+07,5.542386e+07,5.542386e+07,5.542348e+07,5.542348e+07,5.542386e+07
mean,8.077921e+00,-3.875290e+01,1.937328e+01,-3.875261e+01,1.937341e+01,1.685380e+00
std,2.055127e+01,2.556202e+01,1.414234e+01,2.553839e+01,1.412191e+01,1.327664e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396368e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


In [18]:
#Q1. How can there be a negative number for fare_amount? - drop them. 

    # side note : %d
    # %d is a placeholder for a number (while %s is a placeholder for a string)
#how many rows originally
print('Old size: %d' %len(df))

#drop the negative fare_amount
df = df[df.fare_amount >= 0]

#how many rows after dropping negative fare_amount
print('New size: %d' %len(df))

Old size: 55423856
New size: 55421402


## Passenger_count

In [19]:
#Q2. max number of passenger is 208? - check the top 50 by passenger counts

#sort method
#df.sort_values('passenger_count').head(50)

#nlargest method - a lot faster. as it doesn't need to do sorting explicitly
df.nlargest(50, 'passenger_count')

#after 50, still all 208 passengers??

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
929022,3.300000,2009-07-30 11:54:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2154045,3.300000,2010-12-16 11:21:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2198549,3.300000,2010-12-15 14:20:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2910347,4.500000,2010-12-16 06:44:00+00:00,0.000000,0.000000,0.000000,0.000000,208
3107489,2.700000,2009-05-12 14:50:00+00:00,-73.937820,40.758259,-73.937828,40.758251,208
4103745,23.700001,2010-12-22 12:11:00+00:00,0.000000,0.000000,0.000000,0.000000,208
4432483,11.100000,2009-05-11 13:56:00+00:00,-73.937737,40.758266,-73.937737,40.758274,208
7001143,2.700000,2009-02-26 08:29:00+00:00,-73.937874,40.758331,-73.937836,40.758369,208
9310100,3.300000,2012-03-08 10:44:00+00:00,0.000000,0.000000,0.000000,0.000000,208
11153029,39.099998,2012-04-23 07:03:00+00:00,0.000000,0.000000,0.000000,0.000000,208


In [20]:
#how many are above 10 people passenger? 
df[df.passenger_count > 10 ]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
929022,3.3,2009-07-30 11:54:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2154045,3.3,2010-12-16 11:21:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2198549,3.3,2010-12-15 14:20:00+00:00,0.000000,0.000000,0.000000,0.000000,208
2910347,4.5,2010-12-16 06:44:00+00:00,0.000000,0.000000,0.000000,0.000000,208
3107489,2.7,2009-05-12 14:50:00+00:00,-73.937820,40.758259,-73.937828,40.758251,208
...,...,...,...,...,...,...,...
51175550,2.7,2009-05-12 13:18:00+00:00,-73.937881,40.758228,-73.937889,40.758194,208
51978691,3.3,2010-12-14 13:15:00+00:00,0.000000,0.000000,0.000000,0.000000,208
52003599,2.7,2009-03-23 21:52:00+00:00,-73.937866,40.758404,-73.937866,40.758396,208
53368487,2.9,2009-04-14 12:53:00+00:00,-73.937927,40.758263,-73.937904,40.758263,208


In [21]:
#There are 69 rows that are above 10 passenger. Let's get rid of them 
#before
print('Old size: %d' %len(df))

#exclude the passengers more than 10
df = df[df.passenger_count < 11]

#after
print('New size: %d' %len(df))

Old size: 55421402
New size: 55421333


## Null

In [22]:
#Q3. Any null? 
df.isnull().sum()

fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude    376
dropoff_latitude     376
passenger_count        0
dtype: int64

In [23]:
#drop all NAs. 

#before 
print('Old size: %d' %len(df))

#drop NAs
df = df.dropna(how = 'any', axis = 'rows')

#after
print('New size: %d' %len(df))

Old size: 55421333
New size: 55420957


## Geo coordinates

In [28]:
#Q4. do all the coordinates make sense? 
#longitude range: -90 to 90 / longitude range: -180 to 180
#see if any coordinate out of this
print(len(df[(df.pickup_longitude >180) | (df.pickup_longitude < -180)]))
print(len(df[(df.dropoff_longitude >180) | (df.dropoff_longitude < -180)]))
print(len(df[(df.pickup_latitude >90) | (df.pickup_latitude < -90)]))
print(len(df[(df.dropoff_latitude >90) | (df.dropoff_latitude < -90)]))

0
0
0
0


In [25]:
#get rid of the ones from above
#before
print('Old size: %d' %len(df))

#get rid of ones outside of range
df = df[(df.pickup_longitude <= 180) & (df.pickup_longitude >= -180)&(df.dropoff_longitude <= 180) & (df.dropoff_longitude >= -180)&(df.pickup_latitude <= 90) & (df.pickup_latitude >= -90)&(df.dropoff_latitude <= 90) & (df.dropoff_latitude >= -90)] 

#after
print('New size: %d' %len(df))

Old size: 55420957
New size: 55418213


In [ ]:
#after... to check.
print(len(df[(df.pickup_longitude >180) | (df.pickup_longitude < -180)]))
print(len(df[(df.dropoff_longitude >180) | (df.dropoff_longitude < -180)]))
print(len(df[(df.pickup_latitude >90) | (df.pickup_latitude < -90)]))
print(len(df[(df.dropoff_latitude >90) | (df.dropoff_latitude < -90)]))

In [26]:
#minimum
print('Minimum Pickup Longtitude is: %s' %df.pickup_longitude.min())
print('Minimum Dropoff Longtitude is: %s' %df.dropoff_longitude.min())
print('Minimum Pickup Latitude is: %s' %df.pickup_latitude.min())
print('Minimum Dropoff Latitude is: %s' %df.dropoff_latitude.min())

#maximum
print('Maximum Pickup Longtitude is: %s' %df.pickup_longitude.max())
print('Maximum Dropoff Longtitude is: %s' %df.dropoff_longitude.max())
print('Maximum Pickup Latitude is: %s' %df.pickup_latitude.max())
print('Maximum Dropoff Latitude is: %s' %df.dropoff_latitude.max())

Minimum Pickup Longtitude is: -171.79734802246094
Minimum Dropoff Longtitude is: -173.95762634277344
Minimum Pickup Latitude is: -77.83387756347656
Minimum Dropoff Latitude is: -74.3546142578125
Maximum Pickup Longtitude is: 169.97276306152344
Maximum Dropoff Longtitude is: 169.97276306152344
Maximum Pickup Latitude is: 89.7421646118164
Maximum Dropoff Latitude is: 89.81666564941406


In [29]:
#NYC gps coordinate is: Lat: 40.741895 and Long: -73.989308 so something is off about above numbers
df[df.pickup_longitude <-78]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
98652,10.50,2011-05-03 07:45:00+00:00,-78.083328,40.705727,-74.005318,40.728592,1
210060,3.15,2010-07-07 15:15:00+00:00,-121.391212,38.504444,-121.391251,38.504429,3
367368,4.10,2010-10-19 00:39:00+00:00,-78.650909,41.037525,-78.659447,40.971840,1
411093,4.50,2012-08-06 11:25:00+00:00,-80.616669,40.780685,-73.970535,40.789158,2
429196,13.00,2013-03-13 18:14:00+00:00,-78.669426,43.213963,-78.669426,43.213963,1
...,...,...,...,...,...,...,...
55203539,7.30,2012-06-29 10:50:00+00:00,-92.199997,40.773121,-73.959877,40.763699,1
55260154,13.30,2011-12-04 16:48:00+00:00,-87.996002,47.958843,-73.953056,40.788559,1
55363580,8.10,2012-07-09 22:04:00+00:00,-92.233330,40.768604,-73.954948,40.779205,2
55379147,14.50,2013-07-21 18:08:00+00:00,-82.384224,38.148434,-73.954887,40.769314,1
